## Usando RAG para Docs de condomínio


In [1]:
# imports simples

import os
import glob
from dotenv import load_dotenv
import gradio as gr

# imports for langchain, plotly and Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings


In [2]:
# Definindo modelos e banco de dados

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [3]:
# Carregando variáveis de ambiente

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [4]:
# 1. Primeiro, verifique se o caminho está correto
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../0_base_conhecimento"))
CACHE_DIR = os.path.join(BASE_DIR, "processed_docs_cache")

knowledgebase_path = CACHE_DIR
print(f"Verificando o diretório: {knowledgebase_path}")
print(f"Conteúdo do diretório: {os.listdir(knowledgebase_path)}")

# 2. Modifique para procurar tanto em processed_docs_cache quanto em suas subpastas
folders = glob.glob(os.path.join(knowledgebase_path, "*")) + [knowledgebase_path]

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    if os.path.isdir(folder):  # Só processa se for um diretório
        doc_type = os.path.basename(folder)
        print(f"Processando pasta: {folder} (tipo: {doc_type})")
        
        # Tente carregar tanto arquivos .txt na pasta raiz quanto em subpastas
        loader = DirectoryLoader(folder, glob="**/*.txt", loader_cls=TextLoader, 
                               loader_kwargs=text_loader_kwargs, recursive=True)
        folder_docs = loader.load()
        print(f"Encontrados {len(folder_docs)} documentos nesta pasta")
        documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

if not documents:
    print("Nenhum documento foi carregado. Verifique:")
    print(f"1. O caminho {knowledgebase_path} existe?")
    print("2. Existem arquivos .txt no diretório ou subdiretórios?")
else:
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    print(f"Total number of chunks: {len(chunks)}")
    print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Verificando o diretório: /mnt/c/Users/willi/Documents/Projetos/rag_condominio/0_base_conhecimento/processed_docs_cache
Conteúdo do diretório: ['049b078c-b5bf-4d97-bdf7-a2fbd7c66f3b.txt', '065b78d0-c521-4ff9-9932-3825b4b82437.txt', '080a5ffd-2464-40f4-92c0-b4b07e45f617.txt', '2789187e-d466-448c-95ff-f7ff1250a5a4.txt', '280c2644-d328-40d5-bd2c-755a12df2a78.txt', '2ab34b5f-907f-4a18-b0d1-00b152a94df0.txt', '2d264460-0427-482b-ae8d-1c63b20fc53a.txt', '4ef8fafa-78b6-4f4e-8c81-6ca0132fe3b4.txt', '5f64b310-fb6b-412e-b295-96c52a2a1cf4.txt', '5f6bce90-49ed-4d61-858a-fba2f2e1bfe0.txt', '633681a6-5e09-4894-b648-7204c60baea2.txt', '65acae7c-a59f-4e10-8d41-22632869a150.txt', '6917e5df-e366-4198-98b0-1ee290786cb9.txt', '798a79eb-8cbc-4645-9610-474411733ed5.txt', '7cf9f6a5-99d3-4c15-9197-bfb22d2ea64e.txt', '874d4d25-d861-48a4-8060-b2c281d0ff20.txt', '8a71943b-ebd6-45a5-856b-5ead72b58d06.txt', '98e667e9-9dc1-4e00-a6fe-51fb7fa0b1a1.txt', '9c5d36c6-bda9-40df-a782-6f42537facf5.txt', '9c7408f2-660f-4f7d-8

Created a chunk of size 1160, which is longer than the specified 1000
Created a chunk of size 2119, which is longer than the specified 1000
Created a chunk of size 1586, which is longer than the specified 1000
Created a chunk of size 2610, which is longer than the specified 1000
Created a chunk of size 1825, which is longer than the specified 1000
Created a chunk of size 1969, which is longer than the specified 1000
Created a chunk of size 4090, which is longer than the specified 1000
Created a chunk of size 4563, which is longer than the specified 1000
Created a chunk of size 3047, which is longer than the specified 1000
Created a chunk of size 1381, which is longer than the specified 1000
Created a chunk of size 4421, which is longer than the specified 1000
Created a chunk of size 1301, which is longer than the specified 1000
Created a chunk of size 1222, which is longer than the specified 1000
Created a chunk of size 4010, which is longer than the specified 1000
Created a chunk of s

Encontrados 30 documentos nesta pasta


Created a chunk of size 2047, which is longer than the specified 1000


Total number of chunks: 346
Document types found: {'processed_docs_cache'}


In [5]:
# Create embeddings using OpenAI's text-embedding-3-small model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 346 documents


In [6]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 346 vectors with 1,536 dimensions in the vector store


In [7]:
# create a new Chat with OpenAI
llm = ChatOpenAI(
    temperature=0.7,
    model_name=MODEL
)

# Alternative - if you'd like to use Ollama locally, uncomment this line instead
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/tmp/ipykernel_4523/4249715902.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [8]:
sample_questions = [
    # 📝 Atas e Reuniões
    "Quando ocorreu a última assembleia ou reunião registrada do condomínio?",
    "Liste as datas das cinco assembleias ou reuniões mais recentes, em ordem cronológica.",
    "Quais foram os tópicos principais discutidos na assembleia mais recente do condomínio?",
    "Quem são os atuais membros do conselho fiscal, conforme os documentos mais recentes?",
    "De acordo com a última ata de assembleia, qual é o valor atual da taxa condominial definida para os condôminos?",

    # 📄 Contratos
    "Quais contratos ativos o condomínio possui atualmente e com quais empresas foram firmados?",
    "Qual é o valor mensal acordado no contrato de prestação de serviços manutenção de elevadores?",
    "Há cláusulas com penalidades previstas para rescisão antecipada de algum contrato? Se sim, quais?",
    "Quando foi firmado o contrato mais recente e qual é o prazo de vigência previsto?",
    "Existe algum contrato relacionado à manutenção predial, elevadores ou segurança eletrônica? Qual o conteúdo principal?"
]

# 9. Execução das perguntas
for question in sample_questions:
    result = conversation_chain.invoke({"question": question})
    print(f"❓ Pergunta: {question}\n✅ Resposta: {result['answer']}\n")

❓ Pergunta: Quando ocorreu a última assembleia ou reunião registrada do condomínio?
✅ Resposta: A última assembleia registrada do condomínio ocorreu no dia 27 de novembro de 2024.

❓ Pergunta: Liste as datas das cinco assembleias ou reuniões mais recentes, em ordem cronológica.
✅ Resposta: As datas das cinco assembleias ou reuniões mais recentes, em ordem cronológica, são:

1. 04 de novembro de 2024
2. 10 de setembro de 2024
3. 08 de novembro de 2023 (não mencionada diretamente, mas inferida a partir do contexto)
4. 05 de outubro de 2023 (não mencionada diretamente, mas inferida a partir do contexto)
5. 01 de setembro de 2023 (não mencionada diretamente, mas inferida a partir do contexto) 

Nota: As últimas três datas foram inferidas e não estão explicitamente mencionadas no texto fornecido.

❓ Pergunta: Quais foram os tópicos principais discutidos na assembleia mais recente do condomínio?
✅ Resposta: Os tópicos principais discutidos na assembleia recente do condomínio foram:

A) Apres

In [ ]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## Now we will bring this up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [ ]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

In [ ]:
# Let's investigate what gets sent behind the scenes

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

query = "Quando ocorreu a última assembleia ou reunião registrada do condomínio?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)

In [ ]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": 25})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)